In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
# wine.head()

In [4]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42)

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42)

In [8]:
print(data.shape, train_input.shape, test_input.shape)
print(sub_input.shape, val_input.shape)

(6497, 3) (4872, 3) (1625, 3)
(3654, 3) (1218, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9978106185002736
0.8571428571428571


## 3-폴드 교차검증

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) #(모델, 전체데이터 80%, 타겟)
print(scores)

{'fit_time': array([0.01556659, 0.01605296, 0.00897551, 0.00701165, 0.00699854]), 'score_time': array([0.00455046, 0.00498176, 0.00299454, 0.00199771, 0.00601959]), 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}


In [13]:
import numpy as np
np.mean(scores['test_score'])

np.float64(0.8546818301479492)

In [15]:
# 정렬된 데이터 섞어서 사용
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True)
scroes = cross_validate(dt, train_input, train_target, cv=splitter)
scores

{'fit_time': array([0.01556659, 0.01605296, 0.00897551, 0.00701165, 0.00699854]),
 'score_time': array([0.00455046, 0.00498176, 0.00299454, 0.00199771, 0.00601959]),
 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}

## 그리드 서치
- 하이퍼파라미터(파라미터 값 임의 변경)

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
params = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}


In [18]:
dt = DecisionTreeClassifier(random_state= 42)
gs = GridSearchCV(dt, params)

In [19]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
gs.best_estimator_

DecisionTreeClassifier(min_impurity_decrease=0.0003, random_state=42)

In [22]:
gs.best_params_

{'min_impurity_decrease': 0.0003}

In [23]:
gs.cv_results_

{'mean_fit_time': array([0.0071425 , 0.00519605, 0.00400257, 0.00420508, 0.00459552]),
 'std_fit_time': array([1.18260277e-03, 3.97232640e-04, 3.00445252e-06, 3.96028248e-04,
        4.90388361e-04]),
 'mean_score_time': array([0.00291271, 0.00220327, 0.00260191, 0.00219412, 0.00200515]),
 'std_score_time': array([9.19142670e-04, 4.01984596e-04, 4.87690420e-04, 4.01838241e-04,
        3.61390978e-06]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value=1e+20),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.87384615, 0.87076923, 0.87282051, 0.86461538, 0.86051282]),
 'split1_test_score': array([0.86666667, 0.86871795, 0.87794872, 0.88512821, 0.87794872]),
 'split2_test_score': array([0.88603696, 

In [25]:
import numpy as np

params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10),
}

In [27]:
dt = DecisionTreeClassifier(random_state=42)

gs = GridSearchCV(dt, params, n_jobs=-1) # n_jobs = cpu사용

In [28]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
gs.best_params_

{'max_depth': 15,
 'min_impurity_decrease': np.float64(0.0001),
 'min_samples_split': 22}

In [32]:
gs.cv_results_['mean_test_score']

array([0.85837161, 0.85837161, 0.85837161, ..., 0.86309693, 0.86309693,
       0.86309693])

### 랜덤 서치

- randint(범위).rvs(숫자) 정수 데이터 범위 안의 랜덤한 숫자 중에 숫자만큼 균등하게 출력
- uniform 실수데이터

In [33]:
from scipy.stats import uniform, randint

In [34]:
randint(0, 10).rvs(10)

array([1, 6, 5, 2, 5, 3, 8, 6, 2, 3])

In [37]:
rgen = randint(0, 10)
np.unique(rgen.rvs(1000), return_counts=True) # return_counts 데이터의 갯수 

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([113, 109,  91, 106,  77,  95, 111, 109,  96,  93]))

In [38]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25),
}

In [39]:
from sklearn.model_selection import RandomizedSearchCV

In [40]:
gs = RandomizedSearchCV(dt, params, n_iter=100, n_jobs=-1, random_state=42) #n_iter 랜덤한 숫자를 얼마나 반복할지
gs.fit(train_input, train_target)

C:\Users\2-12\Desktop\DMF\ML\venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A703A741A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001A716EC7EF0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A71701C5F0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A71701C830>},
                   random_state=42)

In [41]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=42,
                       min_impurity_decrease=np.float64(0.00037864646423661145),
                       random_state=42)

In [42]:
gs.best_params_

{'max_depth': 42,
 'min_impurity_decrease': np.float64(0.00037864646423661145),
 'min_samples_leaf': 1,
 'min_samples_split': 2}